**CODE BY: NIYATI SINHA**

**ROLL NO: 1805501**

**DARTH VANDER**

######In this notebook where ever I did some mistake or anything, mosty I have just commented that part of code rather than removing it.

######Please read the headings & comments to not what is being done by that block of code.

#**Importing Libraries and Data-Set**

In [2334]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import re

#**Overview Of the DATA SET**

In [ ]:
""" *****************************Data Dictionary**********************

1. business_code : company code of the account
2.cust_number : customer number given to all the customers of the Account
3.name_customer : name of the customer
4.clear_date : The date on which the customer clears an invoice, or in simple terms, they
make the full payment
5.buisness_year :
6.doc_id : It is also an unique identifier of an invoice is a primary key for
acct_doc_header table
7.posting_date :
8.document_create_date : The date on which the invoice document was created
#                       * The date on which the document was created
9.document_create_date.1 :
# due_in_date : The date on which the customer is expected to clear an invoice
# invoice_currency : The currency of the invoice amount in the document for the invoice
# document type : It represents the type of document. eg D1 represents Invoice
# posting_id key : Indicator to identify whether an AR item is invoice, deduction, credit
memo based on its value. Apllicable for SAP ERP
# area_business : Business area in sap is defined as an organisationalarea within the financial
accounting module
# total_open_amount : The amount that is yet to be paid for that invoice
# baseline_create_date : The date on which the Invoice was created
# cust_payment_terms : Business terms and agreements between customers and accounts on
discounts and days of payment
# invoice_id : Unique number assigned when a seller creates an Invoice
# isOpen : Indicator of whether an invoice is open or closed. isopen = 1, means the invoice is open
"""

#**Read Data Set**

In [ ]:
#Reading 1805501.csv from My Drive and storing it as a dataframe
df=pd.read_csv(r'/content/drive/My Drive/Colab Datasets/1805501.csv')
df

**Know your Data Set**

In [ ]:
# Returns the first 5 rows of the dataframe.
df.head()

In [ ]:
# Returns the last 5 rows of the dataframe.
df.tail()

In [ ]:
#Details of each column of the DataFrame df
#DataFrame.info() : returns information about a DataFrame including the index dtype and column dtypes, non-null values and memory usage."""
df.info()

In [ ]:
#Number of invoices that are open and closed
#invoice open means payment is yet to be done
temp = df.groupby('isOpen')['isOpen'].count().to_dict()
print(f"No. of closed invoices : {temp[0]}")
print(f"No. of open invoices : {temp[1]}")

In [ ]:
#Number of invoices have clear_date = null
#Obviously it should be same as number of invoices that are open, still checking it to avoid any case of anomaly. 
print(f"Invoices having \"clear_date\" as Nan : {df.loc[:, 'clear_date'].isna().sum()}")

#**DATA PREPROCESSING AND SANITY CHECK**

###*Conversion of dates to datetime objects*

In [ ]:
"""
df['clear_date'] = pd.to_datetime(df['clear_date'], format='%Y%m%d %H:%M:%S')
df['posting_date'] = pd.to_datetime(df['posting_date'], format='%Y-%m-%d')
df['document_create_date'] = pd.to_datetime(df['document_create_date'], format='%Y-%m-%d')
df['document_create_date.1'] = pd.to_datetime(df['document_create_date.1'], format='%Y-%m-%d')
df['due_in_date'] = pd.to_datetime(df['due_in_date']//1, format='%Y-%m-%d')
df['baseline_create_date'] = pd.to_datetime(df['baseline_create_date']//1, format='%Y-%m-%d')
"""

In [ ]:
print(df.columns) #This returns tuple 
print(type(df.columns)) #pandas index object
print("\n*****************************************************************\n")
print(df.columns.values) #This returns list of column labels of the dataFrame
print(type(df.columns.values)) #numpy array

In [ ]:
#column.values method returs an array of index.
print(pd.Series((df.columns.values)))

In [ ]:
#Series.str.contains() function is used to test if pattern or regex is contained within a string of a Series or Index.
#Creating a list of Columns that contain data in their column label
print(pd.Series(list(df.columns)).loc[lambda x : x.str.contains("date")])

In [ ]:
dateFormatString = ["%Y-%m-%d", "%Y-%m-%d", "%Y%m%d", "%Y%m%d", "%Y%m%d", "%Y%m%d"]
ColumnHeadersList = list(pd.Series(list(df.columns.values)).loc[lambda x : x.str.contains("date")])
#print(ColumnHeaders)
#print(type(df['isOpen'])) # <class 'pandas.core.series.Series'>
index_dateFormatString = 0
for colName in ColumnHeadersList:
    df[colName] = pd.to_datetime(df[colName].astype(str), format = dateFormatString[index_dateFormatString]).copy()
    index_dateFormatString = index_dateFormatString + 1

df.head() 

##*Sanity Check*

In [ ]:
#Details of each column of the DataFrame df
#DataFrame.info() : returns information about a DataFrame including the index dtype and column dtypes, non-null values and memory usage."""
df.info()

In [ ]:
#checking for columns containg null values
df.isnull().any()

In [ ]:
#column headers in form of list which contain null values
df.columns[df.isna().any()].tolist()
#returns the list of those columns which contails atleast one NaN value

In [ ]:
#Finding how many datavalues in column invoice_id is null
df['invoice_id'].isnull().sum()

In [ ]:
#Finding how many datavalues in column area_business is null
df['area_business'].isnull().sum()

######*Dealing with column 'area_business'*

In [ ]:
df['area_business'].unique()

*Dropping Area_business*

In [ ]:
#As all Values are null then there is no significance of this column -> 'area_business'
#drop column
df.pop('area_business')
df.head()

######*Dealing with null values for column 'invoice_id'*

In [ ]:
#as only 6 entries in 50000 are having invoice_id as null thus we can drop these columns to remove any case of ambiguity
#df.dropna is used to remove rows and columns with null values, by subset I have mentioned it to drop those rows which have null values in the column invoice_id
df=df.dropna(subset=['invoice_id'])

In [ ]:
#Details of each column of the DataFrame df
#DataFrame.info() : returns information about a DataFrame including the index dtype and column dtypes, non-null values and memory usage."""
df.info()

#**Train, Test, Validation Split**

####*Spliting Train & Test on the basis of clear_date*

In [ ]:
dfTest = df[df.loc[:, "clear_date"].isna() == True] #clear_date column value is null means payment to be made
dfcopyTest=dfTest.copy
dfTrain = df[df.loc[:, "clear_date"].isna() == False] #clear_date column value is non-null means payment already done

In [ ]:
dfcopyTest

####*Test DataFrame*

In [ ]:
#DataFrame.info() : returns information about a DataFrame including the index dtype and column dtypes, non-null values and memory usage."""
dfTest.info()

In [ ]:
dfTest.shape

In [ ]:
dfTest.head()

####*dfTrain DataFrame*

This will be further split into:
1.   dfTrain
2.   dfVal



In [ ]:
#DataFrame.info() : returns information about a DataFrame including the index dtype and column dtypes, non-null values and memory usage."""
dfTrain.info()

In [ ]:
dfTrain.shape

In [ ]:
dfTrain.head()

####*Spliting dfTrain into Train DataSet and Validation DataSet*


In [ ]:
"""For splitting into train and val2 
splitting should be done on the basis of date as it will include the trends w.r.t time of occurence and season"""


"""On the basis of Dataset values it is not possible to clearly differentiate among the columns 'posting_date','document_create_date','document_create_date.1' and 'baseline_create_date'
so first check that whether they are duplicates or not."""

######*Comparing columns 'posting_date', 'document_create_date', 'document_create_date.1' and 'baseline_create_date' are duplicate or not*

In [ ]:
#df temp is a temporary DataFrame made out of only four columns of the train DataSet
dfTemp = dfTrain[["document_create_date.1", "baseline_create_date", "document_create_date", "posting_date"]]
print(dfTemp)
dfTemp = dfTemp.T #DataFrame.T is done for transpose
print(dfTemp)

In [ ]:
"""DataFrame.duplicated() checks for duplicate values and returns boolean values,
 return false if it is the first occurance and 
 return true if it is the duplicate occurance (other than first occurance)"""

temp = dfTemp.duplicated().to_dict()
print(temp,end='\n\n')
for key in temp:
    print(f"{key:22} : Not Duplicate") if temp[key] == False else print(f"{key:22} : Duplicate")

dfTemp = dfTemp.T # again transposing dfTemp

*Drop Duplicate Column*

In [ ]:
dfTemp

In [ ]:
#Now dropping Duplicate Column
dfTemp.pop('posting_date')
dfTrain.pop('posting_date')
dfTest.pop('posting_date')
dfTrain.head()

In [ ]:
dfTrain.shape

In [ ]:
dfTemp.columns

*Check Correlation*

In [ ]:
"""To find that according to which DataFrame we should split the train and validation dataset
checking how related are these three columns : 
1: document_create_date.1, 
2: baseline_create_date, 
3: document_create_date """

In [ ]:
"""The describe() method is used for calculating some statistical data like percentile, mean and standard deviation
 of the numerical values of the Series or DataFrame. 
It analyzes both numeric and object series and also the DataFrame column sets of mixed data types."""
dfTemp.describe()

In [ ]:
#Correlation cannot be found on datetime objects so instead of using date we can use day number by choosing a specific startdate
#choosing the earliest first date among all three columns
#To see first date among all three columns refer the cell above where we have used the df.describe() 

In [ ]:
startDate= datetime.datetime(2018, 11, 19) #from baseline_create_date column
print(startDate)

In [ ]:
dfTemp['dayNumber_document_create_date.1'] = (dfTemp['document_create_date.1'] - startDate).dt.days
dfTemp['dayNumber_document_create_date'] = (dfTemp['document_create_date'] - startDate).dt.days
dfTemp['dayNumber_baseline_create_date'] = (dfTemp['baseline_create_date'] - startDate).dt.days
dfTemp.head()

In [ ]:
#Calculating Correlation
print("Correlation Matrix : ")
dfTemp[["dayNumber_document_create_date.1", "dayNumber_document_create_date", "dayNumber_baseline_create_date"]].corr()

In [ ]:
#All correlation Matrix values are approximately 1, thus all three columns are very highly correlated
#Thus futher spliting into train and vaidation can be done by choosing any one of the three columns
#1: document_create_date.1, 2: baseline_create_date, 3: document_create_date

#I am choosing document_create_date.1 for further spliting

#splitting data into train and validation in 88% and 12% respectively
from sklearn.model_selection import train_test_split
dfTrain, dfValidation = train_test_split(dfTrain, test_size=0.15)

*Training DataSet*

In [ ]:
dfTrain.describe()

In [ ]:
dfTrain.shape

In [ ]:
dfTrain.head()

In [ ]:
#DataFrame.info() : returns information about a DataFrame including the index dtype and column dtypes, non-null values and memory usage."""
dfTrain.info()

In [ ]:
dfValidation.shape

####*Resetting index of all DataFrames*


In [ ]:
dfTrain.reset_index(drop = True, inplace = True)
dfValidation.reset_index(drop = True, inplace = True)
dfTest.reset_index(drop = True, inplace = True)

In [ ]:
dfTrain.head()

In [ ]:
dfValidation.head()

In [ ]:
dfTest.head()

#**EDA**

###**Exploratory data analysis**

####*Checking columns for unique elements to know the constant features*

In [ ]:
dfTrain.columns

In [ ]:
dfTrain.business_code.unique()
#mixed feature

In [ ]:
dfTrain.cust_number.unique()
#mixed feature

In [ ]:
dfTrain.name_customer.unique()
#string

In [ ]:
dfTrain.buisness_year.unique()
#float

In [ ]:
dfTrain.doc_id.unique()
#double

In [ ]:
dfTrain.invoice_currency.unique()
#currency -> str

In [ ]:
#dfTrain.document type.unique() -> error as label contains space
df['document type'].unique()

In [ ]:
dfTrain.total_open_amount.unique()

In [ ]:
dfTrain.cust_payment_terms.unique()

In [ ]:
dfTrain.invoice_id.unique()

In [ ]:
dfTrain.isOpen.unique()

In [ ]:
dfTrain.posting_id.unique()
#float

#**Feature Engineering**

####*Handling categorical features*

######*Identifying Mixed features*


In [ ]:
dfTrain.info()
#We have to identify mixed features from columns having Dtype = object as all others are specified of specific Dtype

'''
Columns to consider for mixed features:
 0   business_code  
 1   cust_number 
 2   name_customer   
 9   invoice_currency  
 10  document type
 13  cust_payment_terms    '''

 #to check for mixed values check for unique() values in these columns done under the sub-header 'Checking columns for unique elements to know the constant features' of EDA


In [ ]:
#Mixed features means features containing more than one dataType
dfTrain.head()

In [ ]:
#by refering to the unique values in the identified columns mentioned in the first comment of this sub heading
#mixed variable clearly visible: business_code, cust_payment_terms 
#mixed varable not so clearly visible: cust_number

######*business_code*

In [ ]:
dfTrain["business_code"].unique()
#first char part either U or CA
#second int part

In [ ]:
def businessCodeDealing(dataf):
    #seperate string part from business_code and replace U by 0 and CA by 1(for one hot encoding to be implemented)

    dataf.loc[dataf["business_code"].str.contains("^U") == True , "business_code_str_part"] = 0
    dataf.loc[dataf["business_code"].str.contains("^CA") == True , "business_code_str_part"] = 1

    dataf["business_code"] = dataf["business_code"].apply(lambda x : int(x[2:]))
    dataf.rename(columns = {"business_code" : "business_code_int_part"}, inplace = True)
    return dataf

In [ ]:
dfTrain = businessCodeDealing(dfTrain.copy())
dfTest = businessCodeDealing(dfTest.copy())
dfValidation = businessCodeDealing(dfValidation.copy())

In [ ]:
dfTrain.head()

In [ ]:
dfTest.head()

In [ ]:
dfValidation.head()

One Hot Encoding

In [ ]:
#oneHotEncoding the business_code_string_Part
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore')

In [ ]:
enc_df = pd.DataFrame(encoder.fit_transform(dfTrain[["business_code_str_part"]]).toarray())
enc_df = enc_df.rename(columns = {0 : "business_code_str_part_U", 1 : "business_code_str_part_CA"})
dfTrain = dfTrain.join(enc_df).drop(columns = ["business_code_str_part"])

In [ ]:
def encoding(dataf, encoder):
    enc_df = pd.DataFrame(encoder.transform(dataf[["business_code_str_part"]]).toarray())
    enc_df = enc_df.rename(columns = {0 : "business_code_str_part_U", 1 : "business_code_str_part_CA"})
    dataf = dataf.join(enc_df).drop(columns = ["business_code_str_part"])
    return dataf

In [ ]:
dfValidation = encoding(dfValidation, encoder)
dfTest = encoding(dfTest, encoder)

#oneHotEncoding the business_code_string_Part Successfull

In [ ]:
dfTrain.head()

In [ ]:
dfTest.head()

In [ ]:
dfValidation.head()

######*cust_payment_terms*

In [ ]:
dfTrain["cust_payment_terms"].unique()
#sometimes set of characters (alpha) followed by int
#sometimes all characters (alpha)
#sometimes int part followed by set of characters

In [ ]:
"""column 'cust_payment_terms' cannot be segregated into integers and alphabetical part
thus one hot encoding seems to be not possible
so, encode it directly"""

In [ ]:
#Done in the sub Heading Mean Encoding cust_payment_terms later

######*invoice_currency*

In [ ]:
dfTrain["invoice_currency"].unique()

In [ ]:
#Calculating percentage of train set that uses USD and CAD as currency
print(f' USD = {(dfTrain[dfTrain["invoice_currency"] == "USD"].shape[0] / dfTrain.shape[0])*100} % of total dfTrain')
print(f' CSD = {100- (dfTrain[dfTrain["invoice_currency"] == "USD"].shape[0] / dfTrain.shape[0])*100} % of total dfTrain')

In [ ]:
#As the data is split into significant amounts of data thus the lower frequesncy of CSD cannot be ignored
#It would have been possible to ignore it if it was somewhere arund 1% or even lesser
print(f' No of data-entries having USD as invoice_currency {dfTrain[dfTrain["invoice_currency"] == "USD"].shape[0]}')
print(f' No of data-entries having CAD as invoice_currency {dfTrain[dfTrain["invoice_currency"] == "CAD"].shape[0]}')

In [ ]:
dfTrain.head()

In [ ]:
#As my data shows most of the  transactions are done in USD so using USD as Standard mode of payment
##2019's Average exchange rate : 1 CAD = 0.753598 USD
#	Canadian Dollar to US Dollar exchange rate as on Sunday 31 January 2021	is 1 CAD = 0.78102 USD

def currencyConverter(dataf):
    #converting CAD to USD

    #converting total_open_amount to USD wherever invoice_currency is 'CAD'
    dataf.loc[dataf.loc[:, "invoice_currency"] == "CAD", "total_open_amount"] = dataf.loc[dataf.loc[:, "invoice_currency"] == "CAD", "total_open_amount"].apply(lambda x : 0.78102 * x)
    
    #after converting total_open_amount to USD wherever invoice_currency is 'CAD' changing the invoice_currenct to USD
    dataf.loc[dataf.loc[:, "invoice_currency"] == "CAD", "invoice_currency"] = "USD"
    return dataf

In [ ]:
dfTrain = currencyConverter(dfTrain.copy())
dfTest = currencyConverter(dfTest.copy())
dfValidation = currencyConverter(dfValidation.copy())

In [ ]:
dfTrain.invoice_currency.unique(), dfTest.invoice_currency.unique(), dfValidation.invoice_currency.unique()
#constant feature

In [ ]:
dfTrain.head()

######*Mean Encoding cust_payment_terms*

In [ ]:
#creating a mapper function to map "cust_payment_terms" column's values to respective mean
temp = dfTrain[["cust_payment_terms", "total_open_amount"]]
meanMapper = temp.groupby("cust_payment_terms")["total_open_amount"].mean().to_dict()
print(meanMapper)
meanMapper = {key : round(value, 2) for key, value in meanMapper.items()}
print(meanMapper)

#meanMapper is the mapper dictionary

In [ ]:
def meanEncoding(dataf):
    dataf.loc[:, "cust_payment_terms"] = dataf.loc[:, "cust_payment_terms"].map(lambda x : meanMapper.get(x, 0))
    return dataf.copy()


In [ ]:
dfTrain = meanEncoding(dfTrain)
dfTest = meanEncoding(dfTest)
dfValidation = meanEncoding(dfValidation)

#mean encoding the cust_payment_terms

In [ ]:
pd.set_option('display.max_columns', None)

dfTrain.head()

In [ ]:
dfTrain.shape

######*document type*

In [ ]:
temp = dfTrain.copy()
temp["document type"].unique()
#constant feature as far as train dataset is considered

#**EDA**  ( continued )

In [ ]:
dfTrain.head()

In [ ]:
temp=dfTrain.copy()
temp.describe()

In [ ]:
temp["clear_date_month"] = temp["clear_date"].dt.month
temp["clear_date_year"] = temp["clear_date"].dt.year
temp["doc_create_date_month"] = temp["document_create_date.1"].dt.month
temp["doc_create_date_year"] = temp["document_create_date.1"].dt.year

temp["paid_on_or_before_due_date"] = np.where(temp.apply(lambda x : x["clear_date"] <= x["due_in_date"], axis = 1) == True, 1, 0)

In [ ]:
temp.head()

In [ ]:
temp["payment_delay_days"] = temp["clear_date"].sub(temp["due_in_date"]).dt.days.copy()

In [ ]:
sns.set(rc={'figure.figsize':(15,12)})
sns.heatmap(temp.corr(), annot = True)

###*Constant Feature Removal*

In [ ]:
"""Constant features are the type of features that contain only one value for all the outputs in the dataset.
 Constant features provide no information that can help in classification of the record at hand.
Therefore, it is advisable to remove all the constant features from the dataset."""

#Removing constant features as they don't provide any useful information to a machine learning model to predict a target feature.

In [ ]:
def constFeatureCheck(dataf):
    print("No. of unique values in each feature :\n")
    constFeatures = []
    for item in dataf.columns.values:
        uniqueItems = len(dataf[item].unique())
        print(f"{item:22} : {uniqueItems} ")
        if(uniqueItems == 1):
            constFeatures.append(item)
    return constFeatures

constFeaturesDataTrain = constFeatureCheck(dfTrain)
print(f"\nList of constant features in dataTrain: {constFeaturesDataTrain}")

#### Removing constant columns of train set from all the datasets

In [ ]:
dfTrain = dfTrain.drop(columns = constFeaturesDataTrain).copy()
dfValidation = dfValidation.drop(columns = constFeaturesDataTrain).copy()
dfTest = dfTest.drop(columns = constFeaturesDataTrain).copy()

In [ ]:
dfTrain.head()

In [ ]:
dfTrain.shape

### *Quasi - Constant Removal*

In [ ]:
"""Quasi-constant features, as the name suggests, are the features that are almost constant. 
In other words, these features have the same values for a very large subset of the output"""

In [ ]:
#The features having very small variance values have very negligible use for a machine learning model
#to predict the target feature
#thus removing them

In [ ]:
def quasiConstFeatureCheck(dataf, threshold = 1.0):
    print("No. of unique values in each feature :\n")
    quasiConstFeatures = []
    for item in dataf.columns.values:
        uniqueItems = len(dataf[item].unique())
        percentage = (uniqueItems / dataf.shape[0]) * 100
        print(f"{item:28} : {uniqueItems:5} : {percentage:.2f}%")
        if(percentage <= threshold):
            quasiConstFeatures.append(item)
    return quasiConstFeatures

#considering those features as quasi-constant which have less than 0.1% of distinct records
quasiConstFeaturesDataTrain = quasiConstFeatureCheck(dfTrain, 0.1)
print(f"\nList of quasi-constant features in dataTrain:\n{quasiConstFeaturesDataTrain}")

In [ ]:
#Out of these 4 features 'business_code_int_part', 'buisness_year', 'business_code_str_part_U', 'business_code_str_part_CA'
#only 'buisness_year' seems suitable to be dropped


######Removing Quasi-Constant columns of train set from all the datasets

In [ ]:
dfTrain = dfTrain.drop(columns = "buisness_year").copy()
dfValidation = dfValidation.drop(columns = "buisness_year").copy()
dfTest = dfTest.drop(columns = "buisness_year").copy()

In [ ]:
dfTrain.head()

In [ ]:
dfTrain.shape

In [ ]:
dfTrain.columns

###*Checking for duplicate columns*

In [ ]:
#From observation: Possible groups of duplicate features can be:
#1. ("doc_id", "invoice_id")
#2. ("document_create_date.1", "baseline_create_date", "document_create_date")
"""
but group 2 has already been checked 
under the heading 'Comparing columns 'posting_date', 'document_create_date', 'document_create_date.1' and 'baseline_create_date' are duplicate or not' 
with "posting_date"

So, we need to check only group1

"""


####Group1

In [ ]:
#Checking for 1st group:
temp = dfTrain[["doc_id", "invoice_id"]]
#print(temp)

temp = temp.T
#print(temp)

zz = temp.duplicated().to_dict()
print(zz)

for key in zz:
    print(f"{key:22} : Not Duplicate") if zz[key] == False else print(f"{key:22} : Duplicate")

temp = temp.T # again transposing dfTemp

#####Removing duplicate column invoice_id

In [ ]:
dfTrain = dfTrain.drop(columns = "invoice_id").copy()
dfValidation = dfValidation.drop(columns = "invoice_id").copy()
dfTest = dfTest.drop(columns = "invoice_id").copy()

In [ ]:
dfTrain.shape, dfValidation.shape, dfTest.shape

####Group2

In [ ]:
"""but group 2 has already been checked 
under the heading 'Comparing columns 'posting_date', 'document_create_date', 'document_create_date.1' and 'baseline_create_date' are duplicate or not' 
with "posting_date"

There we have found that all three columns 'document_create_date', 'document_create_date.1' and 'baseline_create_date' are different

And all of them are highly coorelated

And we used column document_create_date.1 for splitting thus we can drop the other two
"""

######Removing highly correlated features

In [ ]:
highlyCorrelated = ["baseline_create_date", "document_create_date"]
dfTrain = dfTrain.drop(columns = highlyCorrelated).copy()
dfValidation = dfValidation.drop(columns = highlyCorrelated).copy()
dfTest = dfTest.drop(columns = highlyCorrelated).copy()

In [ ]:
dfTrain.head()

In [ ]:
dfTrain.shape

#**Feature Engineering** ( continued )

In [ ]:
temp.shape

In [ ]:
temp = dfTrain.copy()

#payment_delay_date = clear_date - due_in_date
#after how many days of the due_in_date the customer made the payment
temp["payment_delay_days"] = temp["clear_date"].sub(temp["due_in_date"]).dt.days.copy()
#if payment delay days in negative it means that customer made the payment before the due date
#Similarly if payment delay days in positive it means that customer made the payment after the due date


#payment orig_payment_window_days = due_in_date - document_create_date.1
#after how many days of the invoice document create date we have the due date
temp["orig_payment_window_days"] = temp["due_in_date"].sub(temp["document_create_date.1"]).dt.days.copy()
temp["Total_no_days_taken"]=temp["clear_date"].sub(temp["document_create_date.1"]).dt.days.copy()


#dfValidation["Total_no_days_taken"]=dfValidation["clear_date"].sub(dfValidation["document_create_date.1"]).dt.days.copy()

In [ ]:
temp.head()

In [ ]:
temp.shape

In [ ]:
temp[temp["payment_delay_days"] <= 0].shape, temp[temp["payment_delay_days"] > 0].shape

In [ ]:
temp[(temp["Total_no_days_taken"] >= 200) | (temp["total_open_amount"] >= 300000)].shape

In [ ]:
temp = temp[(temp["Total_no_days_taken"] < 160) & (temp["total_open_amount"] < 300000)]
temp.head()

In [ ]:
""" The different Ageing buckets will be :
● 0-15 days
● 16-30 days
● 31-45 days
● 46-60 days
● Greater than 60 days"""

In [ ]:
bin = [x * 15 for x in range(0,10,1)]
bin

In [ ]:
plt.figure(figsize=(10,6), dpi=80)
n, bins, patches = plt.hist(temp["Total_no_days_taken"], bins = bin, rwidth = 0.9, facecolor='g', alpha=0.75)
plt.xlabel("Total days taken to pay")
plt.ylabel("No. of invoices cleared")
plt.xticks(bin)
plt.grid(True)
plt.show()

In [ ]:
#This means majority of invoices are cleared within 40 days of creation

In [ ]:
sns.set(rc={'figure.figsize':(12,10)})

In [ ]:
sns.heatmap(temp.corr(), annot = True)

In [ ]:
temp['create_month'] = pd.DatetimeIndex(temp['document_create_date.1']).month 
temp['create_weekDay'] = pd.DatetimeIndex(temp['document_create_date.1']).weekday 
temp['create_day'] = pd.DatetimeIndex(temp['document_create_date.1']).day
temp['due_in_month'] = pd.DatetimeIndex(temp['due_in_date']).month 
temp['due_in_weekDay'] = pd.DatetimeIndex(temp['due_in_date']).weekday
temp.head()

In [ ]:
dfValidation['create_month'] = pd.DatetimeIndex(dfValidation['document_create_date.1']).month 
dfValidation['create_weekDay'] = pd.DatetimeIndex(dfValidation['document_create_date.1']).weekday 
dfValidation['create_day'] = pd.DatetimeIndex(dfValidation['document_create_date.1']).day
dfValidation['due_in_month'] = pd.DatetimeIndex(dfValidation['due_in_date']).month 
dfValidation['due_in_weekDay'] = pd.DatetimeIndex(dfValidation['due_in_date']).weekday
dfValidation.head()

In [ ]:
dfTest['create_month'] = pd.DatetimeIndex(dfTest['document_create_date.1']).month 
dfTest['create_weekDay'] = pd.DatetimeIndex(dfTest['document_create_date.1']).weekday 
dfTest['create_day'] = pd.DatetimeIndex(dfTest['document_create_date.1']).day
dfTest['due_in_month'] = pd.DatetimeIndex(dfTest['due_in_date']).month 
dfTest['due_in_weekDay'] = pd.DatetimeIndex(dfTest['due_in_date']).weekday
dfTest.head()

In [ ]:
temp['payment_delay_days'].describe()

In order to see how is the distribution we can use sb.distplot to plot it

In [ ]:
temp.corr().loc['payment_delay_days',:]

In [ ]:
sns.distplot(temp['payment_delay_days'])

In [ ]:
plt.figure(figsize=(20,5))
sns.lineplot(x=temp['document_create_date.1'], y=temp['payment_delay_days'])
plt.show()

In [ ]:
sns.lineplot(x=temp['payment_delay_days'],y=temp['total_open_amount'])
plt.show()

In [ ]:
sns.scatterplot(temp['due_in_weekDay'],temp['payment_delay_days'])
plt.show()

In [ ]:
sns.barplot(x=temp['document_create_date.1'], y=temp['payment_delay_days'])
plt.show()

In [ ]:
corr=temp.corr()
plt.figure(figsize=(20,10))
sns.heatmap(corr,annot=True)
plt.show()

In [ ]:
temp.columns

In [ ]:
tui_df = temp.drop(columns = ["cust_number", "name_customer"]).copy()
tui_df.columns

In [ ]:
tui_df.head()

In [ ]:
#tui_df["payment_delay_days"] = tui_df["clear_date"].sub(tui_df["due_in_date"]).dt.days.copy()
#tui_df["orig_payment_window_days"] = tui_df["due_in_date"].sub(tui_df["document_create_date.1"]).dt.days.copy()

In [ ]:
tui_df_Y = tui_df["payment_delay_days"]
tui_df_X = tui_df.drop(columns = ["payment_delay_days", "clear_date", "document_create_date.1", "due_in_date"]).copy()

In [ ]:
dfValidation["Total_no_days_taken"]=dfValidation["clear_date"].sub(dfValidation["document_create_date.1"]).dt.days.copy()
tui_val1_df = dfValidation.drop(columns = ["cust_number", "name_customer"]).copy()


tui_val1_df["total_open_amount"] = tui_val1_df["total_open_amount"].map(lambda x : round(x, 2))
tui_val1_df["payment_delay_days"] = tui_val1_df["clear_date"].sub(tui_val1_df["due_in_date"]).dt.days.copy()
tui_val1_df["orig_payment_window_days"] = tui_val1_df["due_in_date"].sub(tui_val1_df["document_create_date.1"]).dt.days.copy()

tui_val1_df_Y = tui_val1_df["payment_delay_days"]
tui_val1_df_X = tui_val1_df.drop(columns = ["payment_delay_days", "clear_date", "document_create_date.1", "due_in_date"]).copy()

In [ ]:
dfTest.head()

In [ ]:
#from datetime import date
dfTest["Total_no_days_taken"]=dfTest["due_in_date"].sub(dfTest["document_create_date.1"]).dt.days.copy()

tui_test1_df = dfTest.drop(columns = ["cust_number", "name_customer"]).copy()


tui_test1_df["total_open_amount"] = tui_test1_df["total_open_amount"].map(lambda x : round(x, 2))
tui_test1_df["orig_payment_window_days"] = tui_test1_df["due_in_date"].sub(tui_test1_df["document_create_date.1"]).dt.days.copy()

tui_test1_df_X = tui_test1_df.drop(columns = ["clear_date", "document_create_date.1", "due_in_date"]).copy()


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [ ]:
MSE_Score = []
R2_Score = []
Algorithm = []

In [ ]:
X_train = tui_df_X.copy()
y_train = tui_df_Y.copy()
X_test = tui_val1_df_X.copy()
y_test = tui_val1_df_Y.copy()
X_finaltest = tui_test1_df_X.copy()

In [ ]:
X_test.head()

In [ ]:
X_train.head()

In [ ]:
y_test.head()

In [ ]:
y_train.head()

In [ ]:
X_train.sort_index(inplace=True,axis=1)
X_test.sort_index(inplace=True,axis=1)
X_finaltest.sort_index(inplace=True,axis=1)


In [ ]:
X_train.head()

In [ ]:
X_test.head()

In [ ]:
X_finaltest.head()

In [ ]:
### **1. Linear Regression**

# Fitting Simple Linear Regression to the Training Set
Algorithm.append('Linear Regression')
clf = LinearRegression()
clf.fit(X_train, y_train)

# Predicting the Test Set Results
predicted = clf.predict(X_test)

# Appending the Scores For Visualisation at a Later Part
MSE_Score.append(mean_squared_error(y_test, predicted))
R2_Score.append(r2_score(y_test, predicted))

### **2. Support Vector Regression**

# Fitting SVR to the Training Set
Algorithm.append('Support Vector Regression')
clf = SVR()
clf.fit(X_train, y_train)

# Predicting the Test Set Results
predicted = clf.predict(X_test)

# Appending the Scores For Visualisation at a Later Part
MSE_Score.append(mean_squared_error(y_test, predicted))
R2_Score.append(r2_score(y_test, predicted))

### **3. Decision Tree Regressor**

# Fitting Decision Tree to the Training Set
Algorithm.append('Decision Tree Regressor')
clf = DecisionTreeRegressor()
clf.fit(X_train, y_train)

# Predicting the Test Set Results
predicted = clf.predict(X_test)

# Appending the Scores For Visualisation at a Later Part
MSE_Score.append(mean_squared_error(y_test, predicted))
R2_Score.append(r2_score(y_test, predicted))

### **4. Random Forest Regressor**

# Fitting Random Forest Regressor Tree to the Training Set
Algorithm.append('Random Forest Regressor')
clf = RandomForestRegressor()
clf.fit(X_train, y_train)

# Predicting the Test Set Results
predicted = clf.predict(X_test)

# Appending the Scores For Visualisation at a Later Part
MSE_Score.append(mean_squared_error(y_test, predicted))
R2_Score.append(r2_score(y_test, predicted))

### **5. XGB Regressor**

# Fitting XGBoost Regressor to the Training Set
Algorithm.append('XGB Regressor')
clf = xgb.XGBRegressor()
clf.fit(X_train, y_train)

# Predicting the Test Set Results
predicted = clf.predict(X_test)

# Appending the Scores For Visualisation at a Later Part
MSE_Score.append(mean_squared_error(y_test, predicted))
R2_Score.append(r2_score(y_test, predicted))

## Score Card of the Models' Performances

# Just Combining the Lists into a DataFrame for a Better Visualisation
Comparison = pd.DataFrame(list(zip(Algorithm, MSE_Score, R2_Score)), columns = ['Algorithm', 'MSE_Score', 'R2_Score'])

# Score Card
Comparison

In [ ]:
#choosing Decision Tree Regresser
X_finaltest

In [ ]:
# Fitting Decision Tree to the Training Set
Algorithm.append('Decision Tree Regressor')
clf = DecisionTreeRegressor()
clf.fit(X_train, y_train)

# Predicting the Test Set Results
prediction = clf.predict(X_finaltest)
print(len(prediction))
dfTest['payment_delay_days']=prediction
#dfCopyTest["clear_date"]=(dfCopyTest["due_in_date"])+timedelta(days=dfTest["payment_delay_days"])
#dfcopyTest


In [ ]:
dfTest

In [ ]:
""" The different Ageing buckets will be :
● 0-15 days
● 16-30 days
● 31-45 days
● 46-60 days
● Greater than 60 days"""

In [ ]:
bins=[-60,15,30,45,60,1000]
labels=[0,1,2,3,4]
dfTest['bins']=pd.cut(dfTest['payment_delay_days'],bins=bins,labels=labels,include_lowest=False)

In [ ]:
#import pandas as pd
#dfcopyTest["clear_date"]=(dfcopyTest["due_in_date"])+timedelta(days=dfTest["payment_delay_days"])
#dfcopyTest=pd.dataFrame()
"""
for row in dfcopyTest.iterrows():
  row["clear_date"]=(rows["due_in_date"])+timedelta(days=row["payment_delay_days"])"""

In [ ]:
dfTest

In [ ]:
dfTest.bins.unique()

In [ ]:
dfTest.bins

In [ ]:
len(dfTest.bins.isna()) # all columns are allocated bags